Gemini 2.0 の ハンズオンチュートリアル記事 の実装コード
===================



**1. 概要**：

Google Colaboratory で Gemini 2.0 を動かす、ハンズオンチュートリアルです




<br>

**2. 実装者**：小川雄太郎

**3. 実装日**：2025年01月14日

**4. 実行環境**：Google Colabratory、CPU

**5. 参考記事・スライド類**




ブログ: 「Google Colab で Gemini 2.0 の Multimodal Live API を試す」 [[link](https://note.com/npaka/n/n36ac85db4114)]


ブログ: 「Gemini 2.0 Flash Live API （試験運用版）を使ってAIと日本語で会話してみた」 [[link](https://dev.classmethod.jp/articles/gemini2-flash-exp-live-api/)]


**6. 実装のメイン参考**

https://github.com/GoogleCloudPlatform/generative-ai/tree/main/gemini/getting-started

https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_0_flash.ipynb


https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_chat.ipynb



**7. その他参考にした記事**

公式Doc

https://ai.google.dev/gemini-api/docs/models/gemini-v2

APIリファレンス

https://googleapis.github.io/python-genai/

---

# [0] 初期設定


## [0-1] GPUの初期設定を確認（今回は使用しておらず）


In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## [0-2] installの実施

Gemini 1.5 からは異なり、Gemini 2.0では、新たなパッケージ「Google Gen AI SDK」を使用します

https://github.com/googleapis/python-genai

In [2]:
!pip install google-genai==0.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: google-genai
    Found existing installation: google-genai 0.3.0
    Uninstalling google-genai-0.3.0:
      Successfully uninstalled google-genai-0.3.0


## [0-3] importの実施

In [3]:
# 定番系
import os
import sys
import random
import math
import time
import re
import datetime
import json
import argparse
import numpy as np
import pandas as pd
from pandas.errors import EmptyDataError
from pandas.errors import ParserError
from PIL import Image
from tqdm import tqdm
import gc

# 描画系
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import rc
from ipywidgets import interact
%matplotlib inline

# PyTorch関連
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms

## [0-4] 乱数シードの初期化


In [4]:
# 乱数シードの初期化
seed_val = 1234

print("使用するseedの値：", str(seed_val))
os.environ['PYTHONHASHSEED'] = str(seed_val)
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val) # PyTorchはこちら参考 https://pytorch.org/docs/stable/notes/randomness.html
torch.cuda.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
# GPU高速化
torch.backends.cudnn.benchmark = True

# GPUの再現性まで確保したい場合（ただし、計算速度が低下してしまいます）
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False


使用するseedの値： 1234


## [0-5] 実行環境の確認

In [5]:
# -----------------------------
# 環境情報の確認
# -----------------------------
def print_program_info():
    print("Pythonのバージョン：", sys.version)
    print("---------")
    print("PyTorchのバージョン：",
    torch.__version__)
    print("GPUの枚数：", torch.cuda.device_count())
    global device  # 変数deviceは以降もプログラム本体で使用するのでglobal変数にしています
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)


# 【実行】確認
print_program_info()

# 上記で変数deviceをglobalにしており、deviceでGPUを使用できます


Pythonのバージョン： 3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
---------
PyTorchのバージョン： 2.5.1+cu121
GPUの枚数： 0
cpu


## [0-7] Google Driveのマウント（任意）


In [6]:
# from google.colab import drive
# drive.mount('/content/gdrive')

---

# [1] GeminiのClientを作成

Gemini 2.0 を使用する場合、「Google Cloud の Vertex AI から使用する」、もしくは「単体でAPIキーを発行して使用する」のいずれかになります。


本実装では、「単体でAPIキーを発行して使用する」方法を解説します。

(1) 「Google AI Studio」からAPIキーを取得し、以下の環境変数に代入してください

※簡易版実装なのでAPIキーを直書きしますが、本来はSecretとして安全に利用してください



https://aistudio.google.com/app/apikey

## [1-1] clientをインスタンス化

In [7]:
# clientをインスタンス化

from google import genai

# [1] APIキーの設定
os.environ['GOOGLE_API_KEY'] = "hogehgoe"  # こちらに取得したAPIキーを入力してください

# [2] clientのインスタンス化
client = genai.Client(api_key=os.environ.get('GOOGLE_API_KEY'))


# [2] LLM使用に際する各種設定を実施

## [2-1] model_id

In [8]:
model_id = 'gemini-2.0-flash-exp'

## [2-2] コンテンツセーフティ系の設定

設定内容の詳細は 公式docs [[link](https://ai.google.dev/gemini-api/docs/safety-settings?hl=ja&_gl=1*czyn11*_up*MQ..*_ga*MTE0MzYzNzUxNS4xNzM0NDk2NjQw*_ga_P1DBVKWT6V*MTczNDQ5OTQyMy4yLjAuMTczNDQ5OTQyMy4wLjAuMTg5ODc0NTk3Ng..)] にて。

現在の設定は、「コンテンツが安全でない確率が低い場合、中程度の場合、高い場合にブロックします」です。


In [9]:
from google.genai import types


my_safety_settings = [types.SafetySetting(
    category="HARM_CATEGORY_HATE_SPEECH",
    threshold="BLOCK_LOW_AND_ABOVE"
), types.SafetySetting(
    category="HARM_CATEGORY_DANGEROUS_CONTENT",
    threshold="BLOCK_LOW_AND_ABOVE"
), types.SafetySetting(
    category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
    threshold="BLOCK_LOW_AND_ABOVE"
), types.SafetySetting(
    category="HARM_CATEGORY_HARASSMENT",
    threshold="BLOCK_LOW_AND_ABOVE"
)]



## [2-3] システムプロンプトの設定（system instructions）

In [10]:
system_instruction = """
あなたはユーザーを助けるAgentです。
あなたの名前はHAROです。
"""

## [2-4] 設定内容をGenerateContentConfigにまとめる

In [11]:
my_generate_content_config = types.GenerateContentConfig(
    temperature=0.1,
    top_p=0.95,
    max_output_tokens=2048,
    response_modalities=["TEXT"],
    safety_settings=my_safety_settings,  # [1-4-2] で設定
    system_instruction=[types.Part.from_text(
        system_instruction)],  # [1-4-3] で設定
)


# [3] 会話を1回実行する場合

## [3-1] 入力文を作成

In [12]:
from google.genai.types import (
    Content,
    Part,
)


input_text = "あなたのお名前は？"

my_contents = [Content(role="user", parts=[Part.from_text(input_text)])]

## [3-2] 実行

In [13]:
response = client.models.generate_content(
    model=model_id,  # [1-3-1] で設定
    contents=my_contents,
    config=my_generate_content_config  # [1-3-4]で設定をまとめた
)

print(response.text)


私の名前はHAROです。



In [14]:
response

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='私の名前はHAROです。\n')], role='model'), citation_metadata=None, finish_message=None, token_count=None, avg_logprobs=inf, finish_reason='STOP', grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=[SafetyRating(blocked=None, category='HARM_CATEGORY_HATE_SPEECH', probability='NEGLIGIBLE', probability_score=None, severity=None, severity_score=None), SafetyRating(blocked=None, category='HARM_CATEGORY_DANGEROUS_CONTENT', probability='NEGLIGIBLE', probability_score=None, severity=None, severity_score=None), SafetyRating(blocked=None, category='HARM_CATEGORY_HARASSMENT', probability='NEGLIGIBLE', probability_score=None, severity=None, severity_score=None), SafetyRating(blocked=None, category='HARM_CATEGORY_SEXUALLY_EXPLICIT', probability='

# [4] 会話を往復させる場合

## [4-1] 入力文を作成し、送る

In [16]:
input_text = "私の名前は小川雄太郎です。"

chat = client.chats.create(
    model=model_id,  # [1-3-1] で設定
    config=my_generate_content_config  # [1-3-4]で設定をまとめた
)

# 送り、内容を確認
response = chat.send_message(input_text)
print(response.text)

小川雄太郎さん、こんにちは！HAROです。
どうぞよろしくお願いします。何かお手伝いできることがあれば、遠慮なくおっしゃってくださいね。



In [20]:
# 会話の履歴が残っているかチェック
response = chat.send_message("あれ、私の名前はなんでしたっけ")
print(response.text)

あら、小川雄太郎さん、ご自身の名前を忘れてしまわれたのですか？
先ほど「私の名前は小川雄太郎です」とおっしゃっていましたよ。
もし、何か混乱されているようでしたら、ゆっくりお話を聞かせてくださいね。



## [4-3] 会話履歴のチェック

In [17]:
# 会話履歴のチェック
print(chat.history)

AttributeError: 'Chat' object has no attribute 'history'

In [18]:
# あれ?、上記が通らないので以下で
chat._curated_history


[Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='私の名前は小川雄太郎です。')], role='user'),
 Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='小川雄太郎さん、こんにちは！HAROです。\nどうぞよろしくお願いします。何かお手伝いできることがあれば、遠慮なくおっしゃってくださいね。\n')], role='model')]

# [5] 会話履歴の続きから会話させる場合

## [5-1] 会話履歴の作成

In [21]:
# 履歴をhistoryとして作成
my_history = [
    Content(role="user", parts=[Part.from_text("私の好きな食べ物はみかんです。")]),
    Content(role="model", parts=[Part.from_text("そうなんですか。私はリンゴが好きです")]),
]


## [5-2] 実行

In [22]:
# チャットを作成
chat2 = client.chats.create(
    model=model_id,  # [1-3-1] で設定
    config=my_generate_content_config,  # [1-3-4]で設定をまとめた
    history=my_history  # 上記の作成した会話履歴です
)


In [23]:
# 会話の履歴が残っているかチェック
response = chat2.send_message("あれ、私の好きな食べ物はなんでしたっけ")
print(response.text)

response = chat2.send_message("あなたの好きな食べ物は？")
print(response.text)


あなたの好きな食べ物はみかんです！

私の好きな食べ物はリンゴです！



---

# 以上